In [1]:
!pip install beautifulsoup4

In [9]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd

In [ ]:
def get_links(page_number):
    URL = "https://ielts69.com/essay/p/" + str(page_number)
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, "html.parser")
    results = soup.find(class_="similar-post-list")
    link_elements = results.find_all("a", href=True)
    
    links = []
    
    for link_element in link_elements:
        link_url = link_element["href"]
        links.append(link_url)
    
    return links

In [ ]:
all_links = []
for i in range(1,2061):
    links = get_links(i)
    all_links.append(links)

all_links


In [ ]:
links_array = np.asarray(all_links).ravel()

In [ ]:
df = pd.DataFrame(links_array)
df.to_csv("all_links.csv")

In [21]:
links = pd.read_csv("all_links.csv")


In [22]:
essay_links = links[links['0'].str.startswith('/essay', na=False) == True].reset_index(drop=True)
essay_links

,0
0,/essay/increasing-the-price-of-petrol-is-the-b...
1,/essay/fatherhood-ought-to-be-emphasized-as-mu...
2,/essay/as-computers-are-being-used-more-and-mo...
3,/essay/some-people-believe-that-it-is-good-to-...
4,/essay/some-people-believe-that-teaching-child...
...,...
36235,/essay/tpo-32-do-you-agree-or-disagree-with-th...
36236,/essay/tpo-29-do-you-agree-or-disagree-with-th...
36237,/essay/tpo-22-do-you-agree-or-disagree-with-th...
36238,/essay/tpo-20-do-you-agree-or-disagree-with-th...


In [32]:
list_of_links = np.asarray(essay_links).ravel()

In [64]:
def get_data(page_url):
    new_dict = {}

    URL2 = 'https://ielts69.com' + str(page_url)
    essay_page = requests.get(URL2)
    soup2 = BeautifulSoup(essay_page.content, "html.parser")

    essay_text = soup2.find("div", id="editor")
    #text = results2.find_all()
    
    if essay_text != None:
        for tag in essay_text.select('div.popup'):
            tag.decompose()
    
        delimiter = '#'      

        for line_break in essay_text.findAll('br'):      
            line_break.replaceWith(delimiter)

    paragraph_number = soup2.find("i", id="totalParagraphs")
    word_count = soup2.find("i", id="totalWords")
    essay_topic = soup2.find("h1")
    essay_score = soup2.find("div", id="bandTotal")

    ccTotal = soup2.find("i", id="ccTotal")
    lrTotal = soup2.find("i", id="lrTotal")
    graTotal = soup2.find("i", id="graTotal")
    taTotal = soup2.find("i", id="taTotal")

    new_dict = {'EssayTopic': essay_topic.text, 'EssayBody': "No text" if essay_text == None else essay_text.text, 
            'ParagraphCount': int(paragraph_number.text), 'WordCount': int(word_count.text), 
            'EssayScore': float(essay_score.text), 'CC_Score': float(ccTotal.text), 
            'LR_Score': float(lrTotal.text), 'GRA_Score': float(graTotal.text), 'TA_Score': float(taTotal.text)}
    
    return new_dict

In [65]:
Essay_Dataset = pd.DataFrame(columns=['EssayTopic', 'EssayBody', 'ParagraphCount', 
                                     'WordCount', 'EssayScore', 'CC_Score', 'LR_Score', 
                                     'GRA_Score', 'TA_Score'])

In [66]:
complete = 0
mod = int((len(list_of_links)-26600)/100)
for index_number, url in enumerate(list_of_links[15000:15010]):
    data = get_data(url)
    new_row = pd.DataFrame(data, index=[index_number])
    Essay_Dataset = pd.concat([Essay_Dataset, new_row])
    
    if ((index_number % mod) == 0) and (index_number != 0):
        complete += 1
        print("%{} is complete".format(complete))
        Essay_Dataset.to_csv("essay_dataset_new4-{}.csv".format(complete))
        
print("Done!")

Done!


In [7]:
Essay_Dataset.to_csv("essay_dataset_new.csv")

In [67]:
Essay_Dataset

,EssayTopic,EssayBody,ParagraphCount,WordCount,EssayScore,CC_Score,LR_Score,GRA_Score,TA_Score
0,TPO 54 - Question 4Explain how the example dis...,Some folks believe that punishing children phy...,5,295,7.0,7.0,7.0,7.0,7.0
1,TPO 12 - Q 2Some people believe it's essential...,The majority of employees do not exercise eith...,4,315,7.0,7.0,7.0,7.0,7.0
2,TPO 31 - Question 1Parents need to make sure t...,"The contemporary world, unlike the past, is oc...",4,269,7.0,7.0,7.0,7.0,7.0
3,1055 in some places people spend a lot of mone...,While one half of the population thinks people...,5,254,7.0,7.0,7.0,7.0,7.0
4,The table below shows the results of a survey ...,The public is developing a strong opinion that...,3,413,7.0,7.0,7.0,7.0,7.0
5,Some people say that advertising encourages us...,Some people are commenting that the level of e...,4,371,7.0,7.0,7.0,7.0,7.0
6,Some people think that newspapers are the best...,Every country has different methods for dealin...,4,300,7.0,7.0,7.0,7.0,7.0
7,Some companies have uniforms for their staffs ...,"In the recent years, many people, especially i...",4,351,7.0,7.0,7.0,7.0,7.0
8,Some people think governments should spend mon...,It is a matter of concern that our environment...,4,443,7.0,7.0,7.0,7.0,7.0
9,Question: Some young children spend a great am...,"In this technical world, the profound influenc...",4,270,7.0,7.0,7.0,7.0,7.0


In [68]:
a=list_of_links[15000:15010]

In [69]:
a

array(['/essay/tpo-54-question-4explain-how-the-example-discussed-by-the-professor-illustrates-a-systems-thinking-approach-to-1',
       '/essay/tpo-12-q-2some-people-believe-its-essential-for-a-persons-education-to-learn-to-play-a-musical-instrument-2',
       '/essay/tpo-31-question-1parents-need-to-make-sure-their-children-lead-healthy-lives-1',
       '/essay/1055-in-some-places-people-spend-a-lot-of-money-on-parties-that-celebrate-special-family-events-1',
       '/essay/the-table-below-shows-the-results-of-a-survey-to-find-out-what-members-of-a-city-sports-club-think-about-the-clubs-1',
       '/essay/some-people-say-that-advertising-encourages-us-to-buy-things-that-we-really-do-not-need-25',
       '/essay/some-people-think-that-newspapers-are-the-best-source-of-news-1',
       '/essay/some-companies-have-uniforms-for-their-staffs-which-must-be-worn-all-the-times-1',
       '/essay/some-people-think-governments-should-spend-money-on-faster-means-of-transport-1',
       '/essay/q